In [ ]:
pip install -r requirements.txt

In [1]:
import tensorflow as tf
import tensorflow.keras.layers as tfl
from tensorflow.python.framework import ops
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread
from tqdm import tqdm
import cv2
import os
import imutils

### Image preprocessing ###

The original images contain black backgrounds of various sizes. Feeding these raw images into our model would result in lower accuracy as the model could pickup on background size as a feature for determining tumor type. To combat this, we can crop the images using the cv2 library. 

Below is an example of how we crop images from start to finish, as outlined in crop_image().

|Step 0 ) Original Image      |Step 1) Thresh Image                 |Step 2) Erode Image            | Step 3) Dilate Image         |Step 4) Crop Original Image |
|-----------------------------|-------------------------------------|-------------------------------|------------------------------|----------------------------|
| ![org](utils/original.jpg)  |![thr](utils/after_thresholding.jpg) |![ero](utils/after_eroding.jpg)|![dil](utils/after_dilate.jpg)|![cro](utils/after_crop.jpg)|





In [108]:
def crop_image(img):
    grayscale_image = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    _,thresholded_image = cv2.threshold(grayscale_image, 10, 255, cv2.THRESH_BINARY)
    eroded_image = cv2.erode(thresholded_image, None, iterations=2)
    dilated_image = cv2.dilate(eroded_image, None, iterations=2)
    
    contours = cv2.findContours(dilated_image.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(contours)
    points = max(contours, key=cv2.contourArea) # [[[x_val,y_valval]]]
    
    x_points = []
    y_points = []
    for point in points:
        x_points.append(point[0][0])
        y_points.append(point[0][1])

    left = min(x_points)
    right = max(x_points)
    top = min(y_points)
    bottom = max(y_points)

    cropped_image = img[top:bottom, left:right]
    
    return cropped_image

In [106]:
IMG_SIZE = 400

new_img2 = crop_image(image)
new_img2 = cv2.resize(new_img2,(IMG_SIZE,IMG_SIZE))
cv2.imwrite("test_crop.jpg",new_img2)

True

### References 
https://www.kaggle.com/datasets/masoudnickparvar/brain-tumor-mri-dataset?resource=download

https://github.com/masoudnick/Brain-Tumor-MRI-Classification/blob/main/Preprocessing.py

https://docs.opencv.org/4.x/
                    
https://pyimagesearch.com/2016/04/11/finding-extreme-points-in-contours-with-opencv/